In [1]:
from datasets import load_dataset

dataset = load_dataset("pookie3000/ascii-cats")

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ascii', 'creature'],
        num_rows: 201
    })
})

In [3]:
print(dataset['train'][0]['ascii'])

    /\_/\           ___
   = o_o =_______    \ \ 
    __^      __(  \.__) )
(@)<_____>__(_____)____/


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM,GenerationConfig, TrainingArguments, Trainer
from trl import SFTTrainer

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-1.7B")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-1.7B")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
prompt = """"
{ascii}
"""
EOS_TOKEN = tokenizer.eos_token

def format_dataset(example):
    example['text'] = prompt.format(ascii=example['ascii']) + EOS_TOKEN
    return example

formatted_dataset = dataset.map(format_dataset, remove_columns=['ascii'])

In [4]:
for i in enumerate(formatted_dataset['train'][:3]):
    print(formatted_dataset['train'][i[0]]['text'])

"
    /\_/\           ___
   = o_o =_______    \ \ 
    __^      __(  \.__) )
(@)<_____>__(_____)____/
<|im_end|>
"
|\---/|
| o_o |
 \_^_/
<|im_end|>


In [5]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj","k_proj", "o_proj","gate_proj", "up_proj", "down_proj"],
    lora_dropout=0,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

peft_model = get_peft_model(model, lora_config)

In [6]:
training_args = TrainingArguments(
    output_dir="output",
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=4,
    per_device_train_batch_size=2,
    logging_steps=1,
    lr_scheduler_type="linear",
    optim="adamw_8bit"
)

trainer = SFTTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=formatted_dataset['train'],
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=2048,
    dataset_num_proc=2,
)

trainer.train()

/opt/conda/envs/idm/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, dataset_num_proc. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/envs/idm/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/envs/idm/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:297: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/envs/idm/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argume

Step,Training Loss
1,5.158800
2,4.506300
3,4.207500
4,4.098500
5,4.267700
6,4.161600
7,4.192700
8,3.851600
9,3.749900
10,3.719300


TrainOutput(global_step=104, training_loss=2.3769718021727524, metrics={'train_runtime': 82.5194, 'train_samples_per_second': 9.743, 'train_steps_per_second': 1.26, 'total_flos': 469299953774592.0, 'train_loss': 2.3769718021727524, 'epoch': 4.0})

In [8]:
from transformers import TextStreamer

def generate_ascii_art(model):
    inputs = tokenizer(" ",return_tensors="pt").to("cuda")
    text_streamer = TextStreamer(tokenizer)
    for token in model.generate(**inputs, streamer=text_streamer, max_new_tokens=100):
        print(token)
        pass

generate_ascii_art(peft_model)

  .       .         
\`-"'"-'/
 } o   o {     
=.  T  ,=   
   /^^^\   .
  /       \  )
  |         |
  \         |
  /(  )-(  )\
  ( /`--`--'\ )
/`"_,)   (_,'`\
(,,) ) ) ) (,,)
(,,) ) ) ) (,,)
(,,)
<|im_end|>
tensor([   220,    659,    981,    659,  15677,     59,     63,  27651,  14913,
         22512,   5894,    335,    297,    256,    297,    314,  10947,  45934,
           220,    350,    220,   1154,     28,   5872,    256,    608,  21876,
         24884,    256,  16448,    220,    608,    981,   1124,    220,   1727,
           220,    760,    260,   9248,    220,   1124,    260,   9248,    220,
         70881,    220,    873,   8013,    220,    873,   5661,    220,    320,
           608,     63,    313,     63,    313,  15777,   1727,     14,     63,
             1,   6878,      8,    256,   5453,   2894,     63,   5661,      7,
         10631,      8,    873,    873,    873,    320,  10631,    340,      7,
         10631,      8,    873,    873,    873,    320,  10631,    340